<style>
    .custom-header {
        background-color: #2c3e50;
        color: #ecf0f1;
        padding: 20px;
        text-align: center;
        box-shadow: 0 4px 6px rgba(0, 0, 0, 0.1);
    }
    .custom-header h1 {
        margin: 0;
        font-size: 2.5em;
        font-weight: 300;
    }
</style>

<div class="custom-header">
    <h2>Import Libraries</h2>
</div>

In [2]:
#Data management
import pandas as pd
import numpy as np
np.random.seed(0)
#from pandas_profiling import ProfileReport

#TextBlob Features
from textblob import TextBlob

#Plotting
import matplotlib.pyplot as plt
import plotly.graph_objects as go


#SciKit-Learn
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

#nltk
import nltk
from nltk.corpus import stopwords
#nltk.download('stopwords')
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
#nltk.download('wordnet')

#Tensorflow / Keras
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

#Test
from collections import Counter

<style>
    .custom-header {
        background-color: #2c3e50;
        color: #ecf0f1;
        padding: 20px;
        text-align: center;
        box-shadow: 0 4px 6px rgba(0, 0, 0, 0.1);
    }
    .custom-header h1 {
        margin: 0;
        font-size: 2.5em;
        font-weight: 300;
    }
</style>

<div class="custom-header">
    <h2>Import Data</h2>
</div>

In [3]:
# Train Data
path = "twitter_training.csv"
train_df = pd.read_csv(path, names=["Tweet_ID", "Entity", "Sentiment", "Tweet_Content"])

# Test Data ( Not to be used until the full model has been trained)
test_path = "twitter_validation.csv"
test_df = pd.read_csv(test_path, names=["Tweet_ID", "Entity", "Sentiment", "Tweet_Content"])

In [4]:
train_df

,Tweet_ID,Entity,Sentiment,Tweet_Content
0,2401,Borderlands,Positive,im getting on borderlands and i will murder yo...
1,2401,Borderlands,Positive,I am coming to the borders and I will kill you...
2,2401,Borderlands,Positive,im getting on borderlands and i will kill you ...
3,2401,Borderlands,Positive,im coming on borderlands and i will murder you...
4,2401,Borderlands,Positive,im getting on borderlands 2 and i will murder ...
...,...,...,...,...
74677,9200,Nvidia,Positive,Just realized that the Windows partition of my...
74678,9200,Nvidia,Positive,Just realized that my Mac window partition is ...
74679,9200,Nvidia,Positive,Just realized the windows partition of my Mac ...
74680,9200,Nvidia,Positive,Just realized between the windows partition of...


<style>
    .custom-header {
        background-color: #2c3e50;
        color: #ecf0f1;
        padding: 20px;
        text-align: center;
        box-shadow: 0 4px 6px rgba(0, 0, 0, 0.1);
    }
    .custom-header h1 {
        margin: 0;
        font-size: 2.5em;
        font-weight: 300;
    }
</style>

<div class="custom-header">
    <h2>Dropping NA Values</h2>
</div>

In [5]:
train_df = train_df.dropna()
test_df = test_df.dropna()

In [6]:
train_df.shape

(73996, 4)

<style>
    .custom-header {
        background-color: #2c3e50;
        color: #ecf0f1;
        padding: 20px;
        text-align: center;
        box-shadow: 0 4px 6px rgba(0, 0, 0, 0.1);
    }
    .custom-header h1 {
        margin: 0;
        font-size: 2.5em;
        font-weight: 300;
    }
</style>

<div class="custom-header">
    <h2>Subsetting Data</h2>
</div>

In [7]:
df = train_df.sample(frac=0.1)
df.reset_index(drop=True, inplace=True)

<style>
    .custom-header {
        background-color: #2c3e50;
        color: #ecf0f1;
        padding: 20px;
        text-align: center;
        box-shadow: 0 4px 6px rgba(0, 0, 0, 0.1);
    }
    .custom-header h1 {
        margin: 0;
        font-size: 2.5em;
        font-weight: 300;
    }
</style>

<div class="custom-header">
    <h2>Data Exploration</h2>
</div>

In [8]:
#See overall information about the data frame
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7400 entries, 0 to 7399
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Tweet_ID       7400 non-null   int64 
 1   Entity         7400 non-null   object
 2   Sentiment      7400 non-null   object
 3   Tweet_Content  7400 non-null   object
dtypes: int64(1), object(3)
memory usage: 231.4+ KB


In [9]:
#Display a few entries
df.head()

,Tweet_ID,Entity,Sentiment,Tweet_Content
0,2535,Borderlands,Negative,@ Borderlands. Can we get a big fix please? I'...
1,11073,TomClancysGhostRecon,Negative,@GhostRecon You call 1.1.0 an update. You're h...
2,3024,Dota2,Neutral,I'm excited to watch the EU's closed qualifyin...
3,6719,Fortnite,Negative,"anyways but my fortnite here,, im annoyed."
4,1533,Battlefield,Negative,Collateral with the revolver is so satisfying....


<style>
    .custom-header {
        background-color: #2c3e50;
        color: #ecf0f1;
        padding: 20px;
        text-align: center;
        box-shadow: 0 4px 6px rgba(0, 0, 0, 0.1);
    }
    .custom-header h1 {
        margin: 0;
        font-size: 2.5em;
        font-weight: 300;
    }
</style>

<div class="custom-header">
    <h2>Checking Balance of the in Output Categories</h2>
</div>

In [10]:
# Calculate the counts and proportions of each sentiment
sentiment_counts = df['Sentiment'].value_counts()
sentiments = sentiment_counts.index.tolist()
sentiment_nums = (sentiment_counts / sentiment_counts.sum()).tolist()

# Create the bar chart
fig = go.Figure(data=[go.Bar(x=sentiments, y=sentiment_nums)])

# Update layout for better visualization
fig.update_layout(
    title="Distribution of Sentiment Classes",
    xaxis_title="Sentiment",
    yaxis_title="Proportion",
    yaxis_tickformat=".2%",
    template="plotly_white"
)

# Show the plot
fig.show()

In [11]:
!pip install --upgrade nbformat

